<a href="https://colab.research.google.com/github/milijan/market/blob/main/keras/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import math
import pandas_datareader as web
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

plt.style.use('fivethirtyeight')

In [9]:
df = web.DataReader('AAPL', data_source='yahoo', start='2012-01-01', end='2020-11-21' )
df.shape

(2238, 6)